In [144]:
from __future__ import division, print_function
import numpy as np
from hmmlearn.hmm import MultinomialHMM
import poetrytools
import itertools

In [155]:
%run 'preprocessing.ipynb'

In [156]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [1]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [2]:
reversed_quatrain_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_volta_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_couplet_model = MultinomialHMM(n_components=30, n_iter=20, verbose=True)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

In [149]:
def generate_b_line(seed, rhymes, model, length=7):
    b_line = []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_

    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    possible_start_emissions = np.where(rhymes[seed] == 1)
    probs = np.array(emission_probs[start_state][possible_start_emissions])

    scaled_probs = probs / sum(probs)
    
    start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
    b_line.append(start_emission)
    
    curr_state = start_state
    for _ in range(length - 1):
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
        b_line.append(curr_emission)
        
    return b_line

def convert_a_line(sample, words):
    ret = ''
    reversed_list = []
    for word in np.nditer(sample):
        reversed_list.append(word)
    
    i = 0
    for word in reversed(reversed_list):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def convert_b_line(sample, words):
    ret = ''
    i = 0
    for word in reversed(sample):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def generate_pair(model, rhymes, length=7):
    while True:
        a_line = model.sample(length)[0]
        seed = a_line[0][0]
        if len(np.where(rhymes[seed] == 1)[0]) > 0:
            b_line = generate_b_line(seed, rhymes, model)
            return a_line, b_line
        
def generate_rhyming_sonnet():
    sonnet = ''
    a_lines = []
    b_lines = []
    
    for _ in range(4):
        a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    for i in range(2):
        sonnet += convert_a_line(a_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_a_line(a_lines[2 * i + 1], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_b_line(b_lines[2 * i + 1], quatrain_words) + '\n'
    
    a_lines = []
    b_lines = []
    
    for _ in range(2):
        a_line, b_line = generate_pair(reversed_volta_model, volta_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    sonnet += convert_a_line(a_lines[0], volta_words) + '\n'
    sonnet += convert_a_line(a_lines[1], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[0], volta_words) + '\n'
    sonnet += convert_b_line(b_lines[1], volta_words) + '\n'
    
    a_line, b_line = generate_pair(reversed_couplet_model, couplet_rhymes)
    sonnet += convert_a_line(a_line, couplet_words) + '\n'
    sonnet += convert_b_line(b_line, couplet_words) + '\n'
    
    return sonnet

def generate_10_rhyming_sonnets():
    sonnets = ''
    for i in range(10):
        sonnets += str(i) + '\n' + generate_rhyming_sonnet() + '\n'
    
    f = open("project2data/rhyming_shakespeare.txt","w")
    f.write(sonnets)
    return sonnets

In [146]:
print(generate_10_rhyming_sonnets())

In [141]:
# checks if line is in iambic pentameter (i.e. 0101010101 stress pattern)
def check_iambic_pentameter(line):
    # get the stresses from cmu dict 
    # if word is 1 syllable, then have the option for it to be stressed or unstressed
    stresses = []
    for i in line.split(' '):
        stress = poetrytools.stress(i)
        if len(stress) == 1:
            stresses.append(['0','1'])
        else:
            stresses.append([stress])
    
    # make combination of all possible stress patterns
    result = [[]]
    final = []
    for pool in stresses:
        result = [x+[y] for x in result for y in pool]
    final = [''.join(i) for i in result]
    
    # return if any pattern fits iambic pentameter 
    return ('0101010101' in final)